Short Backwarding for Selecting the Best Historical Agent in a Consulting System for Portfolio Allocation with Deep Reinforcement Learning

## 3.0 Stock Selection
---
Auto Encoders are employed to select the less volatile stocks by choosing stocks with less reconstruction error. These are the stocks that are included in the portfolio

### 3.1 Import Relevant Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

In [2]:
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Activation
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras import regularizers, optimizers

from sklearn import preprocessing

### 3.2 Load the Data

In [3]:
# Load the close prices dataset
prices_data = pd.read_csv('./datasets/close_prices.csv')

In [4]:
df = prices_data.copy()

In [5]:
df = df.reset_index(drop=True).set_index(['date'])

In [6]:
df.head()

,MCD,CVX,DD,PG,UNH,V,WBA,RTX,INTC,VZ,...,AAPL,GS,IBM,TRV,HD,CAT,XOM,DIS,KO,BA
date,,,,,,,,,,,,,,,,,,,,,
2008-03-19,34.153214,43.431351,32.355705,42.133186,28.681099,12.605218,22.749361,29.209345,13.014488,14.425960,...,3.915352,127.940323,64.913177,31.375137,17.912857,47.481430,46.908859,26.417377,18.233225,54.094543
2008-03-20,34.636936,44.131432,32.417919,43.102497,28.495552,14.356561,22.999493,29.315552,13.421770,14.832500,...,4.024055,138.037750,65.684776,32.308086,18.727085,47.539391,47.225548,26.975491,18.561644,55.088791
2008-03-24,35.496170,44.555729,33.466522,43.475304,28.810194,13.325832,24.143843,29.880541,13.656266,15.181546,...,4.213072,137.461456,66.089973,32.240482,19.527952,48.962620,47.753334,27.093884,18.604212,56.186138
2008-03-25,35.769848,44.836826,33.644249,43.208118,28.463276,14.111155,23.768646,29.804058,13.742652,15.152799,...,4.256853,138.037750,65.484924,32.287811,19.194254,49.355450,47.336636,27.127707,18.674158,55.898918
2008-03-26,35.387978,45.059536,33.599819,43.270256,27.511274,14.269550,23.737381,29.570433,13.489651,14.840708,...,4.380049,134.848679,64.896484,32.145817,18.793810,49.819118,47.925571,26.857107,18.595097,56.193501


### 3.3 Define Functions

In [7]:
def defineAutoencoder(num_stock, encoding_dim = 5, verbose=0):
    
    """
    Function for fitting an Autoencoder
    """

    # connect all layers
    input = Input(shape=(num_stock,))

    encoded = Dense(encoding_dim, kernel_regularizer=regularizers.l2(0.00001),name ='Encoder_Input')(input)

    decoded = Dense(num_stock, kernel_regularizer=regularizers.l2(0.00001), name ='Decoder_Input')(encoded)
    decoded = Activation("linear", name='Decoder_Activation_function')(decoded)

    # construct and compile AE model
    autoencoder = Model(inputs=input, outputs=decoded)
    adam = optimizers.Adam(learning_rate=0.0005)
    autoencoder.compile(optimizer=adam, loss='mean_squared_error')
    if verbose!= 0:
        autoencoder.summary()

    return autoencoder

In [8]:
def getReconstructionErrorsDF(df_pct_change, reconstructed_data):
    
    """
    Function for calculating the reconstruction Errors
    """
    array = []
    stocks_ranked = []
    num_columns = reconstructed_data.shape[1]
    for i in range(0, num_columns):
        diff = np.linalg.norm((df_pct_change.iloc[:, i] - reconstructed_data[:, i]))  # 2 norm difference
        array.append(float(diff))

    ranking = np.array(array).argsort()
    r = 1
    for stock_index in ranking:
        stocks_ranked.append([ r
                              ,stock_index
                              ,df_pct_change.iloc[:, stock_index].name
                              ,array[stock_index]
                              ])
        r = r + 1

    columns = ['ranking','stock_index', 'stock_name' ,'recreation_error']
    df = pd.DataFrame(stocks_ranked, columns=columns)
    df = df.set_index('stock_name')
    return df

### 3.4 Get the Percentage Change of the Close Prices

In [9]:
col_names = df.columns.to_list()

In [10]:
print(col_names)

['MCD', 'CVX', 'DD', 'PG', 'UNH', 'V', 'WBA', 'RTX', 'INTC', 'VZ', 'WMT', 'JNJ', 'AXP', 'MRK', 'PFE', 'JPM', 'CSCO', 'NKE', 'MSFT', 'MMM', 'AAPL', 'GS', 'IBM', 'TRV', 'HD', 'CAT', 'XOM', 'DIS', 'KO', 'BA']


In [11]:
df_pct_change = df.pct_change(1).astype(float)
df_pct_change = df_pct_change.replace([np.inf, -np.inf], np.nan)
df_pct_change = df_pct_change.fillna(method='ffill')

# the percentage change function will make the first two rows equal to nan
df_pct_change = df_pct_change.tail(len(df_pct_change) - 2)

C:\Users\nice\AppData\Local\Temp\ipykernel_11248\3045634350.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_pct_change = df_pct_change.fillna(method='ffill')


In [12]:
df_pct_change.shape

(4033, 30)

In [13]:
# remove columns where there is no change over a longer time period
df_pct_change = df_pct_change[df_pct_change.columns[((df_pct_change == 0).mean() <= 0.05)]]

In [14]:
df_pct_change.head()

,MCD,CVX,DD,PG,UNH,V,WBA,RTX,INTC,VZ,...,AAPL,GS,IBM,TRV,HD,CAT,XOM,DIS,KO,BA
date,,,,,,,,,,,,,,,,,,,,,
2008-03-24,0.024807,0.009614,0.032346,0.008649,0.011042,-0.071795,0.049755,0.019273,0.017471,0.023533,...,0.046972,-0.004175,0.006169,-0.002092,0.042765,0.029938,0.011176,0.004389,0.002293,0.019920
2008-03-25,0.007710,0.006309,0.005311,-0.006146,-0.012042,0.058932,-0.015540,-0.002560,0.006326,-0.001894,...,0.010392,0.004192,-0.009155,0.001468,-0.017088,0.008023,-0.008726,0.001248,0.003760,-0.005112
2008-03-26,-0.010676,0.004967,-0.001321,0.001438,-0.033447,0.011225,-0.001315,-0.007839,-0.018410,-0.020596,...,0.028941,-0.023103,-0.008986,-0.004398,-0.020863,0.009394,0.012441,-0.009975,-0.004234,0.005270
2008-03-27,-0.002698,-0.006591,-0.013640,-0.003303,-0.002932,-0.016417,-0.018441,-0.002874,-0.035224,-0.004980,...,-0.033159,-0.041828,-0.011889,-0.001892,-0.013139,-0.004266,-0.000696,-0.011965,0.002616,-0.027261
2008-03-28,0.000541,0.001184,-0.005966,0.000432,0.011765,-0.002384,0.004563,-0.012390,-0.014225,-0.003059,...,0.019679,-0.021946,-0.008224,-0.000842,-0.014753,0.000779,-0.011369,-0.012110,-0.006035,-0.010105


### 3.5 Construct the Autoencoder

In [15]:
# define the input parameters
hidden_layers = 5
batch_size = 300
epochs = 500
stock_selection_number = 20
num_stock = df_pct_change.shape[1]
verbose = 1

In [16]:
print('-' * 20 + 'Step 1 : Returns vs. recreation error (recreation_error)')
print('-' * 25 + 'Transform dataset with MinMax Scaler')

--------------------Step 1 : Returns vs. recreation error (recreation_error)
-------------------------Transform dataset with MinMax Scaler


In [17]:
# Normalize the data
df_scaler = preprocessing.MinMaxScaler()
df_pct_change_normalised = df_scaler.fit_transform(df_pct_change)

In [18]:
# define autoencoder
print('-' * 25 + 'Define autoencoder model')
num_stock = len(df_pct_change.columns)
autoencoder = defineAutoencoder(num_stock=num_stock, encoding_dim=hidden_layers, verbose=verbose)
#plot_model(autoencoder, to_file='img/model_autoencoder_1.png', show_shapes=True,
#           show_layer_names=True)

-------------------------Define autoencoder model


KeyboardInterrupt: 

In [ ]:
# train autoencoder
print('-' * 25 + 'Train autoencoder model')
autoencoder.fit(df_pct_change_normalised, df_pct_change_normalised, shuffle=True, epochs=epochs,batch_size=batch_size,verbose=verbose)

-------------------------Train autoencoder model


Epoch 1/500
14/14 [==============================] - 5s 12ms/step - loss: 0.2599
Epoch 2/500
14/14 [==============================] - 0s 8ms/step - loss: 0.2124
Epoch 3/500
14/14 [==============================] - 0s 10ms/step - loss: 0.1762
Epoch 4/500
14/14 [==============================] - 0s 6ms/step - loss: 0.1485
Epoch 5/500
14/14 [==============================] - 0s 12ms/step - loss: 0.1268
Epoch 6/500
14/14 [==============================] - 0s 10ms/step - loss: 0.1087
Epoch 7/500
14/14 [==============================] - 0s 10ms/step - loss: 0.0931
Epoch 8/500
14/14 [==============================] - 0s 15ms/step - loss: 0.0790
Epoch 9/500
14/14 [==============================] - 0s 14ms/step - loss: 0.0663
Epoch 10/500
14/14 [==============================] - 0s 8ms/step - loss: 0.0549
Epoch 11/500
14/14 [==============================] - 0s 9ms/step - loss: 0.0448
Epoch 12/500
14/14 [==============================] - 0s 12ms/step - loss: 0.0361
Epoch 13/500
14/14 [=========

KeyboardInterrupt: 

In [ ]:
# predict autoencoder
print('-' * 25 + 'Predict autoencoder model')
reconstruct = autoencoder.predict(df_pct_change_normalised)

-------------------------Predict autoencoder model
127/127 [==============================] - 2s 7ms/step


In [ ]:
# Inverse transform dataset with MinMax Scaler
print('-' * 25 + 'Inverse transform dataset with MinMax Scaler')
reconstruct_real = df_scaler.inverse_transform(reconstruct)
df_reconstruct_real = pd.DataFrame(data=reconstruct_real, columns=df_pct_change.columns)

-------------------------Inverse transform dataset with MinMax Scaler


In [ ]:
print('-' * 25 + 'Calculate L2 norm as reconstruction loss metric')
df_recreation_error = getReconstructionErrorsDF(df_pct_change=df_pct_change,
                                                reconstructed_data=reconstruct_real)

-------------------------Calculate L2 norm as reconstruction loss metric


In [ ]:
df_recreation_error

,ranking,stock_index,recreation_error
stock_name,,,
JNJ,1,11,0.433799
PG,2,3,0.495820
KO,3,28,0.527868
PFE,4,14,0.548123
MCD,5,0,0.585698
MMM,6,19,0.600459
VZ,7,9,0.607145
RTX,8,7,0.608574
HD,9,24,0.644845


In [ ]:
filtered_stocks = df_recreation_error.head(stock_selection_number).index

In [ ]:
filtered_stocks
my_filtered_stocks = pd.DataFrame(filtered_stocks)
my_filtered_stocks.to_csv('filtered_stocks.csv')

In [ ]:
# store the list of selected stocks
%store filtered_stocks

Stored 'filtered_stocks' (Index)
